In [1]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, TimeoutException
from selenium.webdriver.chrome.service import Service
import pandas as pd

# Set up the Selenium driver (make sure you have the appropriate web driver executable in your system PATH)
service = Service()
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)

# Navigate to the LinkedIn login page
driver.get("https://www.linkedin.com/login")

# Get LinkedIn username and password from the user
print("Please enter the exact LinkedIn username you use to login (email/phone?):")
username_string = str(input())
print()
print("Please enter the exact LinkedIn password:")
password_string = str(input())
print()

elementID = driver.find_element(By.ID, 'username')
elementID.send_keys(username_string)
elementID = driver.find_element(By.ID, 'password')
elementID.send_keys(password_string)
elementID.submit()

# Wait for login to complete
time.sleep(5)

# Links of posts
links = [
    "https://www.linkedin.com/feed/update/urn:li:activity:7088410019511042048?utm_source=share&utm_medium=member_desktop",
    "https://www.linkedin.com/posts/iamdivyanshu_attacking-the-kubernetes-cluster-divyanshu-activity-7085981967141257216-Yg89?utm_source=share&utm_medium=member_desktop",
    "https://www.linkedin.com/posts/iamdivyanshu_hacktricks-cloud-ctf-activity-7084564282566209537-1uIa?utm_source=share&utm_medium=member_desktop",
    "https://www.linkedin.com/posts/iamdivyanshu_attacking-and-defending-kubernetes-cluster-activity-7080507698496110592-5NFu?utm_source=share&utm_medium=member_desktop",
    "https://www.linkedin.com/feed/update/urn:li:activity:7080024583998763008?utm_source=share&utm_medium=member_desktop",
    "https://www.linkedin.com/feed/update/urn:li:activity:7080139705761054722?utm_source=share&utm_medium=member_desktop",
     "https://www.linkedin.com/posts/iamdivyanshu_github-justmorpheusvery-vulnerable-serverless-activity-7057660481515229184-Pqp6?utm_source=share&utm_medium=member_desktop",
]

commenters = []
urls = []
comments = []

for url in links:
    driver.get(url)
    time.sleep(3)

    try:
        if driver.find_element(By.CLASS_NAME, 'msg-overlay-list-bubble--is-minimized') is not None:
            pass
    except NoSuchElementException:
        try:
            if driver.find_element(By.CLASS_NAME, 'msg-overlay-bubble-header') is not None:
                driver.find_element(By.CLASS_NAME, 'msg-overlay-bubble-header').click()
        except NoSuchElementException:
            pass

    # To load more comments
    while 1:
        try:
            show_more_btn = driver.find_elements(By.XPATH, './/button[contains(@class,"comments-comments-list__load-more-comments-button artdeco-button artdeco-button--muted artdeco-button--1 artdeco-button--tertiary ember-view")]')
            time.sleep(1)
        except:
            show_more_btn = 0
            break
        if show_more_btn != 0:
            time.sleep(2)
            if len(show_more_btn) > 0:
                try:
                    show_more_btn[0].click()
                    time.sleep(1)
                except StaleElementReferenceException:
                    show_more_btn = driver.find_elements(By.XPATH, './/button[contains(@class,"comments-comments-list__load-more-comments-button artdeco-button artdeco-button--muted artdeco-button--1 artdeco-button--tertiary ember-view")]')
                    time.sleep(1)
            else:
                break

    # To load more replies
    driver.execute_script("window.scrollBy(0,-12000);")
    while 1:
        try:
            load_more_btn = driver.find_element(By.XPATH, './/button[contains(@class,"button show-prev-replies t-12 t-black t-normal hoverable-link-text")]/span')
            time.sleep(1)
        except:
            load_more_btn = 0
            break
        if load_more_btn != 0:
            time.sleep(2)
            try:
                load_more_btn.click()
                time.sleep(1)
            except StaleElementReferenceException:
                load_more_btn = driver.find_elements(By.XPATH,'.//button[contains(@class,"button show-prev-replies t-12 t-black t-normal hoverable-link-text")]/span')
                time.sleep(1)
        else:
            break

    time.sleep(5)
    # Here starts the main coding
    comment_elements = driver.find_elements(By.XPATH,'.//div[contains(@class,"comments-post-meta__profile-info-wrapper display-flex")]')

    for person in comment_elements:
        try:
            name = person.find_element(By.XPATH,'.//span[contains(@class,"comments-post-meta__name-text")]').text.strip('\n...')
        except:
            name = 'unknown'            
        try:
            commenter_url = person.find_element(By.XPATH,'.//a[contains(@class,"ember-view inline-flex overflow-hidden t-16 t-black t-bold tap-target")]').get_attribute('href')
        except NoSuchElementException:
            commenter_url = person.find_element(By.XPATH, '//a[contains(@class, "app-aware-link  inline-flex overflow-hidden t-16 t-black t-bold tap-target")]').get_attribute('href')

        commenters.append(name)
        urls.append(commenter_url)

        # Extract the comments
        try:
            comment = person.find_element(By.XPATH, './/div[contains(@class, "comments-post-meta__comment t-14 t-black--light t-normal break-words")]').text
            comments.append(comment)
        except NoSuchElementException:
            comments.append("Comment not found")

# Initialize lists for commenters and URLs
for i, (commenter, url) in enumerate(zip(commenters, urls), 1):
    print(f"Commenter {i}: {commenter}")
    print(f"Commenter URL: {url}")
    print("---")

# Save data to an Excel file
df = pd.DataFrame({
    'Commenter': commenters,
    'Commenter URL': urls
})

file_name = 'linkedin_commenters.xlsx'
df.to_excel(file_name, index=False)

# Quit the driver
driver.quit()


Please enter the exact LinkedIn username you use to login (email/phone?):

Please enter the exact LinkedIn password:

Commenter 1: Anurag Kanere
Commenter URL: https://www.linkedin.com/in/anuragkanere/
---
Commenter 2: RAKESH KUMAR MEHTA
Commenter URL: https://www.linkedin.com/in/rakesh-kumar-mehta-33ab9724/
---
Commenter 3: Kaustubh Gedam
Commenter URL: https://www.linkedin.com/in/kaustubh-gedam-91709a183/
---
Commenter 4: Kunal Mehta
Commenter URL: https://www.linkedin.com/in/kunal-mehta205/
---
Commenter 5: Sujatha Yakasiri
Commenter URL: https://www.linkedin.com/in/sujathayakasiri/
---
Commenter 6: Divyanshu ..
View Divyanshu ..’s profile
Commenter URL: https://www.linkedin.com/in/ACoAAB3rna0BUmMLZB1mxqsU9INy0I2PHOMA0w0
---
Commenter 7: Sheeraz Ali
Commenter URL: https://www.linkedin.com/in/beingsheerazali/
---
Commenter 8: Atul kishor Jaiswal
Commenter URL: https://www.linkedin.com/in/atulkishorjaiswal/
---
Commenter 9: Rushabh Pinesh Mehta, PGP-ITBM, CISM (Q), CDPSE™
Commenter UR